## Content based filtering door natural language processing en clustering

In deze notebook zullen een recommender system bouwen door gebruik te maken van Content based filtering door natural language processing en clustering.


In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.cluster import DBSCAN
from sklearn.cluster import SpectralClustering
import time
import spacy
nlp = spacy.load("en_core_web_md")

def load_jsons(data_path, file):
    """ helper function to load '.json' files (they're not proper jsons) """
    file_path = data_path + file
    with open(file_path) as jsons:
        lines = [json.loads(json_line) for json_line in jsons]
    return pd.DataFrame(lines)

def split_data(data, d = 0.75):
    """Split data in a training and test set.
    
    Arguments:
    data -- any dataFrame.
    d    -- the fraction of data in the training set
    """
    np.random.seed(seed=5)
    mask_test = np.random.rand(data.shape[0]) < d
    return data[mask_test], data[~mask_test]

**We laden een zelfgemaakte pickle bestand in**

We gaan de reviews en businesses gebruiken

In [218]:
reviews = pd.read_pickle('./data/train_reviews.pkl')
reviews_test = pd.read_pickle('./data/test_reviews.pkl')
businesses = pd.read_pickle('./data/businesses.pkl')

display(reviews.head())
display(businesses.head())

businesses.shape

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,qx6WhZ42eDKmBchZDax4dQ,0,2017-03-27 01:14:37,0,Amo5gZBvCuPc_tZNpHwtsA,5.0,"Our family LOVES the food here. Quick, friendl...",1,DzZ7piLBF-WsJxqosfJgtA
2,1XOty79t4o_e3v0OOw8LTA,2,2017-07-28 17:55:02,0,JTvVNbqHbdLq3zeuxtjoug,5.0,We have had a wonderful experience with Dr. Be...,2,qOiMHf9eBZhsvDSqauIiug
4,9Q1ZtzTPFWG4fJiFSko5Xg,0,2016-05-31 05:00:31,0,TetO8IIvbaaX9xnC7ZE1Rw,5.0,My boyfriend and I went here for dinner evenin...,0,CBYlr6bksWM1WA_pTumeeg
6,ipFobdM6X_4SNMA9ZBfdAQ,0,2013-10-24 02:49:09,0,e2YdyDmFaps6frh6kWRABw,4.0,Pita jungle is one of our family's favorite pl...,0,X61CbjwRDWWPRkN61PIpyA
8,ipFobdM6X_4SNMA9ZBfdAQ,1,2014-01-17 03:39:20,1,-x2NjuDTiphKwCixSrv_SQ,4.0,I've eaten at this location twice. The first t...,1,NgavU49JWmusG2BjfqNdaw


,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,"560 E Germann Rd, Ste 107",{'BusinessAcceptsCreditCards': 'True'},hgPhZoB3zdiRdlT9FETskw,"Auto Glass Services, Auto Detailing, Automotiv...",Gilbert,"{'Monday': '0:0-0:0', 'Tuesday': '7:0-18:0', '...",1,33.278251,-111.777460,OzBraz,85297,12,5.0,AZ
1,709 W Ray Rd,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...",GWO87Y-IqL54_Ijx6hTYAQ,"Food, Desserts, Juice Bars & Smoothies, Ice Cr...",Gilbert,"{'Monday': '12:0-21:0', 'Tuesday': '12:0-21:0'...",0,33.320386,-111.804652,Bruster's Real Ice Cream,85233,57,4.5,AZ
2,"4024 E Guadalupe Rd, Ste 101","{'BusinessAcceptsCreditCards': 'True', 'Restau...",KiE81YHZCEdPPlmrXghgzA,"Beauty & Spas, Nail Salons",Gilbert,"{'Monday': '9:0-19:0', 'Tuesday': '9:0-19:0', ...",1,33.365437,-111.704174,Sun Valley Nail and Spa,85234,34,3.5,AZ
5,"2680 S Val Vista Dr. Ste 135, Bldg 7 Ste 135","{'AcceptsInsurance': 'True', 'BusinessAcceptsC...",SVhM3Pxjw-PUPr1sBghqlw,"Diagnostic Services, Diagnostic Imaging, Radio...",Gilbert,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,33.302772,-111.756137,SimonMed Imaging - Gilbert Spectrum,85295,93,2.0,AZ
6,"2487 S Gilbert Rd, Ste 102","{'GoodForKids': 'True', 'BusinessAcceptsCredit...",cmsAFAymfVoBitSF8R6GKw,"Chinese, Restaurants",Gilbert,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",1,33.304830,-111.788866,Yao Fine Chinese Cuisine,85295,200,4.0,AZ


(1797, 14)

**Similarity matrix**

We make eerst een similarity matrix van alle categorien die er zijn. De similarities zijn op basis van de spacy similarities van de woorden.

In [219]:
cats = []
counter = 0
for i in set(businesses.categories.dropna().tolist()):
    for j in i.split(", "):
        if j not in cats:
            cats.append(j)
len(cats)

686

In [220]:
category_similarities = pd.DataFrame(index = cats, columns = cats)
category_similarities.head()

,Restaurants,Sushi Bars,Japanese,Event Planning & Services,Venues & Event Spaces,Parks,Active Life,Tires,Automotive,Auto Repair,...,Roof Inspectors,Beer Gardens,Bus Tours,Airport Shuttles,Tours,Limos,Town Car Service,Taxis,Afghan,Fingerprinting
Restaurants,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sushi Bars,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Japanese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Event Planning & Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Venues & Event Spaces,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [221]:
start = time.time()

def index_column(x):
    return x.index + "|" + x.name

def calc_tokenizer(x):
    x = x.split('|')
    return [nlp.tokenizer(x[0]), nlp.tokenizer(x[1])] 

def calc_spacy_sim(x):
    return x[0].similarity(x[1])

category_similarities = category_similarities.apply(index_column)
category_similarities = category_similarities.applymap(calc_tokenizer)
category_similarities = category_similarities.applymap(calc_spacy_sim)
end = time.time()
print(end - start)

category_similarities.head()

132.95219492912292


,Restaurants,Sushi Bars,Japanese,Event Planning & Services,Venues & Event Spaces,Parks,Active Life,Tires,Automotive,Auto Repair,...,Roof Inspectors,Beer Gardens,Bus Tours,Airport Shuttles,Tours,Limos,Town Car Service,Taxis,Afghan,Fingerprinting
Restaurants,1.000000,0.644762,0.232002,0.370796,0.495505,0.517621,0.159977,0.121777,0.144614,0.185797,...,0.195959,0.517928,0.420175,0.376703,0.395954,0.214480,0.463057,0.357344,-0.008694,0.034089
Sushi Bars,0.644762,1.000000,0.402007,0.263675,0.400182,0.300625,0.192152,0.220439,0.180166,0.185344,...,0.258527,0.501124,0.294847,0.256308,0.245594,0.227127,0.340791,0.285518,-0.026270,0.065389
Japanese,0.232002,0.402007,1.000000,0.144797,0.125722,0.144447,0.162116,0.124427,0.244286,0.242789,...,0.135724,0.275075,0.208976,0.101577,0.159081,0.117148,0.247219,0.220940,0.249511,0.091210
Event Planning & Services,0.370796,0.263675,0.144797,1.000000,0.761293,0.395385,0.520809,0.143130,0.292998,0.406552,...,0.426487,0.366143,0.469243,0.383354,0.436258,0.221360,0.648785,0.301258,0.066591,0.359822
Venues & Event Spaces,0.495505,0.400182,0.125722,0.761293,1.000000,0.540055,0.474777,0.158898,0.228669,0.244329,...,0.368444,0.502188,0.466384,0.349723,0.468227,0.276109,0.511410,0.254735,0.005135,0.278571


**Clustering**

Eerst maken we numpy array en aan de hand daarvan maken we gebruik van een scikit learning clustering algortime. We krijgen meestal de beste resultaten door 16 clusters te kiezen.

In [222]:
category_similarities.values

array([[ 1.        ,  0.64476154,  0.23200245, ...,  0.35734422,
        -0.00869446,  0.03408948],
       [ 0.64476154,  1.        ,  0.40200663, ...,  0.28551821,
        -0.02627024,  0.06538853],
       [ 0.23200245,  0.40200663,  1.        , ...,  0.22094001,
         0.24951142,  0.09121014],
       ...,
       [ 0.35734422,  0.28551821,  0.22094001, ...,  1.        ,
         0.06544518,  0.13344034],
       [-0.00869446, -0.02627024,  0.24951142, ...,  0.06544518,
         1.        ,  0.07394426],
       [ 0.03408948,  0.06538853,  0.09121014, ...,  0.13344034,
         0.07394426,  1.        ]])

In [223]:
cluster_list = SpectralClustering(16).fit_predict(category_similarities.values).tolist()
print(cluster_list)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\cluster\spectral.py:462: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"


[12, 12, 2, 12, 12, 12, 12, 0, 12, 12, 12, 12, 0, 12, 12, 12, 12, 12, 11, 12, 1, 11, 12, 12, 12, 12, 7, 11, 11, 12, 11, 12, 2, 2, 12, 7, 12, 12, 12, 8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 2, 12, 12, 12, 12, 12, 12, 12, 12, 7, 12, 0, 0, 12, 0, 7, 12, 12, 12, 12, 12, 12, 0, 2, 12, 2, 12, 7, 2, 12, 12, 12, 12, 0, 12, 12, 12, 12, 5, 12, 12, 12, 12, 7, 7, 12, 6, 7, 12, 12, 12, 2, 12, 12, 12, 12, 7, 2, 12, 12, 12, 12, 12, 12, 12, 7, 12, 7, 7, 12, 12, 12, 12, 12, 12, 0, 0, 12, 12, 12, 12, 5, 12, 12, 12, 12, 12, 12, 11, 11, 11, 11, 11, 11, 8, 12, 12, 12, 5, 12, 12, 10, 12, 14, 12, 12, 12, 12, 12, 12, 12, 12, 12, 7, 7, 7, 2, 12, 9, 12, 12, 12, 12, 12, 14, 13, 12, 12, 0, 12, 12, 12, 12, 0, 15, 11, 12, 12, 12, 12, 7, 12, 12, 12, 12, 12, 11, 12, 12, 12, 12, 12, 12, 12, 10, 12, 12, 7, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 7, 7, 2, 12, 12, 7, 12, 7, 12, 12, 12, 12, 12, 12, 12, 12, 15, 12, 12, 12, 12, 12, 12

In [224]:
cluster_serie = pd.Series(cluster_list, index=category_similarities.index)

In [225]:
business_categories = businesses[['business_id', 'categories']].copy()
business_categories = business_categories.dropna()
business_categories

,business_id,categories
0,hgPhZoB3zdiRdlT9FETskw,"Auto Glass Services, Auto Detailing, Automotiv..."
1,GWO87Y-IqL54_Ijx6hTYAQ,"Food, Desserts, Juice Bars & Smoothies, Ice Cr..."
2,KiE81YHZCEdPPlmrXghgzA,"Beauty & Spas, Nail Salons"
5,SVhM3Pxjw-PUPr1sBghqlw,"Diagnostic Services, Diagnostic Imaging, Radio..."
6,cmsAFAymfVoBitSF8R6GKw,"Chinese, Restaurants"
...,...,...
3456,LPCAjcX8GOlCDsHMZdsPvg,"Car Stereo Installation, Automotive"
3457,wHn5jKZc3lt_Cu8uoBblgw,"Massage Therapy, Massage, Skin Care, Health & ..."
3458,Mx3atpdnV-cJDcJoZBSjxQ,"Shopping, Fashion, Women's Clothing, Accessories"
3463,_r-OvLO5wqncmsBCg3i0qw,"Restaurants, Italian, Pizza"


**Categorien toevoegen**

Alle categorieen die in hetzelfde cluster zitten worden toegevoegd aan de business

In [226]:
start = time.time()

def get_cluster(x):
    cat = []
    for i in x.split(", "):         
        cat += cluster_serie[cluster_serie == cluster_serie[i]].index.tolist()
    return set(cat)

business_categories.categories = business_categories.categories.map(get_cluster)
business_categories.categories = business_categories.categories.map(lambda x: '|'.join(x))

end = time.time()
print(end - start)

10.884629011154175


In [228]:
def extract_categories(business):
    """Create an unfolded genre dataframe. Unpacks genres seprated by a '|' into seperate rows.

    Arguments:
    movies -- a dataFrame containing at least the columns 'movieId' and 'genres' 
              where genres are seprated by '|'
    """
    genres_m = business.apply(lambda row: pd.Series([row['business_id']] + row['categories'].lower().split("|")), axis=1)
    stack_genres = genres_m.set_index(0).stack()
    df_stack_genres = stack_genres.to_frame()
    df_stack_genres['business_id'] = stack_genres.index.droplevel(1)
    df_stack_genres.columns = ['category', 'business_id']
    return df_stack_genres.reset_index()[['business_id', 'category']]

def pivot_genres(df):
    """Create a one-hot encoded matrix for genres.
    
    Arguments:
    df -- a dataFrame containing at least the columns 'movieId' and 'genre'
    
    Output:
    a matrix containing '0' or '1' in each cell.
    1: the movie has the genre
    0: the movie does not have the genre
    """
    return df.pivot_table(index = 'business_id', columns = 'category', aggfunc = 'size', fill_value=0)

In [229]:
df_categories = extract_categories(business_categories)
# df_categories['category'] = df_categories.categories
# df_categories[df_categories.categories == 'cmsAFAymfVoBitSF8R6GKw']
df_categories

,business_id,category
0,hgPhZoB3zdiRdlT9FETskw,carpet cleaning
1,hgPhZoB3zdiRdlT9FETskw,sushi bars
2,hgPhZoB3zdiRdlT9FETskw,septic services
3,hgPhZoB3zdiRdlT9FETskw,gold buyers
4,hgPhZoB3zdiRdlT9FETskw,car buyers
...,...,...
926116,ABjONdA5Fw8XBOM65tmW4w,chicken shop
926117,ABjONdA5Fw8XBOM65tmW4w,men's hair salons
926118,ABjONdA5Fw8XBOM65tmW4w,apartments
926119,ABjONdA5Fw8XBOM65tmW4w,outdoor gear


**Utility en similarity matrix**

Aan de hand van de nieuw toegevoegde categorien maken een utility matrix en berekenen we aan de hand van de Jaccard index een similarity matrix

In [230]:
df_utility_categories = pivot_genres(df_categories)
display(df_utility_categories.head())

category,& probates,acai bowls,accessories,accountants,acne treatment,active life,acupuncture,adult,adult education,adult entertainment,...,windows installation,windshield installation & repair,wine & spirits,wine bars,wine tasting room,wineries,women's clothing,wraps,yoga,zoos
business_id,,,,,,,,,,,,,,,,,,,,,
--FLdgM0GNpXVMn74ppCGw,1,0,1,0,0,1,1,1,1,1,...,1,1,1,1,1,0,1,0,0,0
-2wfiUn6AZm2KO0FJYakVQ,1,0,1,0,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
-461KOozus5M0Px02uyRgg,1,0,1,0,0,1,1,1,1,1,...,1,1,1,1,1,0,1,0,0,0
-4bpDsuRmpjWjxrtdYfJQA,1,0,1,0,0,1,1,1,1,1,...,1,1,1,1,1,0,1,0,0,0
-6O_cneh21YEljiDhVX78g,1,0,1,0,0,1,1,1,1,1,...,1,1,1,1,1,0,1,0,0,0


In [231]:
def create_similarity_matrix_jaccard(matrix):
    m11 = matrix @ matrix.T
    m10 = pd.DataFrame(matrix.sum(axis = 1).values + np.zeros(m11.shape), index = m11.index, columns = m11.index)
    m01 = m10.T
    return m11/(m01 + m10 - m11)

In [232]:
df_similarity_categories = create_similarity_matrix_jaccard(df_utility_categories)
df_similarity_categories

business_id,--FLdgM0GNpXVMn74ppCGw,-2wfiUn6AZm2KO0FJYakVQ,-461KOozus5M0Px02uyRgg,-4bpDsuRmpjWjxrtdYfJQA,-6O_cneh21YEljiDhVX78g,-6tvduBzjLI1ISfs3F_qTg,-8wrG6WiLkSRS1JgE7jD0Q,-AliE9BXGQl8pIrdlpCOxQ,-I1-KL5RkUcDM3JbYPI_1g,-JzL9Xkx42k82uKSy3X5Zw,...,ze3AgRdWZmneIdpZDmxJuA,zgPrDvk5x0FZj4Wbzb8R6w,zhVrl8N_zMW8f9NuoluG6Q,zniY0Pd9BTUwoFuejSMPBg,zpLOsO7DuxcwwhmHtiASWg,zu1JllEH2cjH6uijHIB6TQ,zvhS7uG_206nRE2ufmmLPQ,zwvshlu1bE2na9sXYrP0TQ,zx0SIkaS1RkqzPFmbUBeFA,zxHN6Rx5MUlVO-qpyuAcLw
business_id,,,,,,,,,,,,,,,,,,,,,
--FLdgM0GNpXVMn74ppCGw,1.000000,0.865149,1.000000,1.000000,1.000000,0.937381,0.911439,0.940952,0.995968,0.940952,...,0.984064,0.859130,0.940952,0.911439,1.000000,0.984064,0.911439,0.911439,1.000000,1.000000
-2wfiUn6AZm2KO0FJYakVQ,0.865149,1.000000,0.865149,0.865149,0.865149,0.817881,0.949212,0.820598,0.862129,0.820598,...,0.853195,0.897351,0.820598,0.949212,0.865149,0.853195,0.949212,0.949212,0.865149,0.865149
-461KOozus5M0Px02uyRgg,1.000000,0.865149,1.000000,1.000000,1.000000,0.937381,0.911439,0.940952,0.995968,0.940952,...,0.984064,0.859130,0.940952,0.911439,1.000000,0.984064,0.911439,0.911439,1.000000,1.000000
-4bpDsuRmpjWjxrtdYfJQA,1.000000,0.865149,1.000000,1.000000,1.000000,0.937381,0.911439,0.940952,0.995968,0.940952,...,0.984064,0.859130,0.940952,0.911439,1.000000,0.984064,0.911439,0.911439,1.000000,1.000000
-6O_cneh21YEljiDhVX78g,1.000000,0.865149,1.000000,1.000000,1.000000,0.937381,0.911439,0.940952,0.995968,0.940952,...,0.984064,0.859130,0.940952,0.911439,1.000000,0.984064,0.911439,0.911439,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zu1JllEH2cjH6uijHIB6TQ,0.984064,0.853195,0.984064,0.984064,0.984064,0.923364,0.898182,0.926829,0.980159,0.926829,...,1.000000,0.847341,0.926829,0.898182,0.984064,1.000000,0.898182,0.898182,0.984064,0.984064
zvhS7uG_206nRE2ufmmLPQ,0.911439,0.949212,0.911439,0.911439,0.911439,0.859130,1.000000,0.862129,0.908088,0.862129,...,0.898182,0.942609,0.862129,1.000000,0.911439,0.898182,1.000000,1.000000,0.911439,0.911439
zwvshlu1bE2na9sXYrP0TQ,0.911439,0.949212,0.911439,0.911439,0.911439,0.859130,1.000000,0.862129,0.908088,0.862129,...,0.898182,0.942609,0.862129,1.000000,0.911439,0.898182,1.000000,1.000000,0.911439,0.911439


In [233]:
def pivot_ratings(df):
    """Creates a utility matrix for user ratings for movies
    
    Arguments:
    df -- a dataFrame containing at least the columns 'movieId' and 'genres'
    
    Output:
    a matrix containing a rating in each cell. np.nan means that the user did not rate the movie
    """
    return df.pivot_table(values='stars', columns='user_id', index='business_id')






def predict_ratings(similarity, utility, to_predict):
    """Predicts the predicted rating for the input test data.
    
    Arguments:
    similarity -- a dataFrame that describes the similarity between items
    utility    -- a dataFrame that contains a rating for each user (columns) and each movie (rows). 
                  If a user did not rate an item the value np.nan is assumed. 
    to_predict -- A dataFrame containing at least the columns movieId and userId for which to do the predictions
    """
    # copy input (don't overwrite)
    ratings_test_c = to_predict.copy()
    # apply prediction to each row
    ratings_test_c['predicted rating'] = to_predict.apply(lambda row: predict_ids(similarity, utility, row['user_id'], row['business_id']), axis=1)
    return ratings_test_c

### Helper functions for predict_ratings_item_based ###

def predict_ids(similarity, utility, userId, itemId):
    # select right series from matrices and compute
    if userId in utility.columns and itemId in similarity.index:
        return predict_vectors(utility.loc[:,userId], similarity[itemId])
    return 0

def predict_vectors(user_ratings, similarities):
    # select only movies actually rated by user
    relevant_ratings = user_ratings.dropna()
    
    # select corresponding similairties
    similarities_s = similarities[relevant_ratings.index]
    
    # select neighborhood
    similarities_s = similarities_s[similarities_s > 0.0]
    relevant_ratings = relevant_ratings[similarities_s.index]
    
    # if there's nothing left return a prediction of 0
    norm = similarities_s.sum()
    if(norm == 0):
        return 0
    
    # compute a weighted average (i.e. neighborhood is all) 
    return np.dot(relevant_ratings, similarities_s)/norm

def mse(predicted_ratings):
    """Computes the mean square error between actual ratings and predicted ratings
    
    Arguments:
    predicted_ratings -- a dataFrame containing the columns rating and predicted rating
    """
    diff = predicted_ratings['stars'] - predicted_ratings['predicted rating']
    return (diff**2).mean()

In [234]:
df_utility_ratings = pivot_ratings(reviews)
df_utility_ratings

user_id,--2HUmLkcNHZp0xw6AMBPg,--3WaS23LcIXtxyFULJHTA,--MOaYz3TyMKVSVoR0cW_w,--Rm-VyYWgHA7ZpljAMlvg,--unw0LLleszob8UnoMVrw,--vrYCvpozoPQ14yNh3ogA,--xUSSxDXPEaE9BGZKtVDQ,-01CztZ8o5UbarmEiQLDsQ,-03VewnKHQJAoxqTqjhDkA,-056p1z8OpIrGon4bgEKNg,...,zz7TmdlPu_rbfunhfSV9OA,zzF17hwmlFTuOa1Yagi-eg,zzO9aMo33jA3pPv8SoYskw,zzOxlK5lm7jlulotNoYd4g,zzRq3Ykz8yfr2mcsPhYnEg,zzWlTfNssZPlQo4_cKujGw,zzY9daJLRVCbcm79qiZqug,zzeO-4J8u1gZS4Vg_J8jyg,zzjQuRnY_Kgr_zaw6A-ACQ,zzkTUnua51enVp7OrWBwNg
business_id,,,,,,,,,,,,,,,,,,,,,
--FLdgM0GNpXVMn74ppCGw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-2wfiUn6AZm2KO0FJYakVQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-461KOozus5M0Px02uyRgg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-4bpDsuRmpjWjxrtdYfJQA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-6O_cneh21YEljiDhVX78g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zu1JllEH2cjH6uijHIB6TQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zvhS7uG_206nRE2ufmmLPQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zwvshlu1bE2na9sXYrP0TQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Predictions**

Met dit algoritme kunnen 19841 van de 30075 ratings voorspeld worden met een MSE van 2.477005223574279

In [240]:
df_sim = predict_ratings(df_similarity_categories, df_utility_ratings, reviews_test[['user_id', 'business_id', 'stars']])
# df_sim_notzero = df_sim[df_sim["predicted rating"] > 0]
# df_sim_notzero["predicted rating"] = df_sim.stars.mean()
df_sim

,user_id,business_id,stars,predicted rating
1,k9lY8q4dvX8BwCE2n4PpBA,Z1wApYkrMmMQinjDdbSLbg,1.0,0.000000
3,W0QVEiIElFQ9ulVrMp9_nA,7tdVsSDXRKtLLC-fD1hJ-A,4.0,0.000000
7,ETWISuSphgjc3puFAplnJg,5GwFUNl6nsDsMuv435G1Eg,5.0,0.000000
18,6EGqHVa7QAVOSx5ufxtzHA,7tdVsSDXRKtLLC-fD1hJ-A,4.0,4.060837
32,VF7jQT9efeFOU38BWqpQ6Q,ZRHSidI7IIeXhTHMc9L_gw,2.0,0.000000
...,...,...,...,...
129194,F_d3CsPwcE964tBdHaSaNg,N5mU4iB5jCy7B4re_i2jIQ,1.0,0.000000
129197,HKao-hyqqkEu6o4rm2Q3Zw,iNkdV6ggGJFa3lYU3fKuqw,2.0,0.000000
129202,oq9Srm2cM93hcED_CMDlPQ,A9hmWmvIIscQKHc3pKb3VA,5.0,0.000000
129204,eUUMw_QfC-ieeqfKVf6UcQ,v0zWuMRsK-e8hsY5VOvdog,3.0,2.996198


In [243]:
mse(df_sim)

7.406326205849982

In [244]:
df_sim_notzero = df_sim[df_sim["predicted rating"] > 0]
df_sim_notzero

,user_id,business_id,stars,predicted rating
18,6EGqHVa7QAVOSx5ufxtzHA,7tdVsSDXRKtLLC-fD1hJ-A,4.0,4.060837
33,vtmHM-EdhVYScgCj-1Ri2g,NXVaLK8uh1twtiuBKHsoQw,1.0,2.256857
39,NLM3hJttpFnXo8EPLbryoA,mXyLqsbe7FT1X8ZFOP8WpQ,5.0,3.613665
45,nJRKUlAXK3evHI9zZYK2zA,mXyLqsbe7FT1X8ZFOP8WpQ,1.0,2.994302
47,ohlQBkcLZo7rbeSXNKmq6A,qx6WhZ42eDKmBchZDax4dQ,5.0,1.000000
...,...,...,...,...
129170,8xTbX-d72wD5lAL1YHuPTw,g-vBADstbU8dIMR_ltN3eA,5.0,5.000000
129182,Fkxphy9sghf1bAqbHEYjaQ,oPkaBoX_0hVIBV_VXp7nlA,5.0,4.654484
129183,uYIc6yHDhJ4VVDxEN5AWfA,iLLzOJLU4tPAN9UUHlyZiQ,5.0,3.647332
129191,8IuxHBypodI4PalectVIjQ,BIIXZBTzTjRclj5G607-qw,5.0,4.757382


In [245]:
mse(df_sim_notzero)

2.477005223574279